# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import DataScienceHelperLibrary as dsh
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
import numpy as np
import pandas as pd
import pickle
import re
import seaborn as sns

from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse

from sklearn.metrics import accuracy_score, precision_score, recall_score, make_scorer

from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.svm import SVC

from sqlalchemy import create_engine


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ben\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ben\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ben\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ben\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ben\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ben\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
class DisasterConfig:
    Database = 'MyDisasterResponse.db'
    Table = 'Message'
    SaveFileJoblib = 'MyDisasterPredictor.pkl'
    SaveFilePickle = 'MyDisasterPredictor.sav'

In [3]:
# load data from database
database = 'MyDisasterResponse.db'
table = 'Message'

engine = create_engine('sqlite:///{}'.format(DisasterConfig.Database))

df = pd.read_sql_table(DisasterConfig.Table, engine)
#df = pd.read_sql("SELECT * FROM {}".format(table), engine)

In [4]:
dsh.AnalyzeDataFrame(df)

-------------------- Dataframe analysis started --------------------
Shape:  (26216, 40)
Number of duplicate rows:  0
-------------------- Analysis of Columns with NaN values --------------------
Columns having all values: 39, 97.50%
id, type: int64                        0.0
message, type: object                  0.0
genre, type: object                    0.0
related, type: int64                   0.0
request, type: int64                   0.0
offer, type: int64                     0.0
aid_related, type: int64               0.0
medical_help, type: int64              0.0
medical_products, type: int64          0.0
search_and_rescue, type: int64         0.0
security, type: int64                  0.0
military, type: int64                  0.0
child_alone, type: int64               0.0
water, type: int64                     0.0
food, type: int64                      0.0
shelter, type: int64                   0.0
clothing, type: int64                  0.0
money, type: int64                 

Hmm... there were nan values that have been inserted into database...

In [5]:
df[df.original == np.nan]

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report


Hmmm.... there were definitely missing values...

In [6]:
type(df.at[7408, 'original']) # example ID I radonmly found.

str

In [7]:
df[df.original == None]

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report


In [8]:
type(None)

NoneType

Hmmm.... I thought that None or np.nan would be interpretet as something like DBNULL. Instead it is a string value?! I didn't calculated that...

In [9]:
df[df.original == 'NoneType']

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report


I give up... I thought there is an adequate handling for np.nan und database NULL values. I just intended to check the number of mising values before and after writing into database. But as you can see, currently I'm out of ideas how to do it ... And I don't intend to waste time with that.

## Remark: Cleaning again
Before I split data, I remove those rows that would be null/none/empty after going through Tokenize() to avoid bad side effects. For example I don't konw how a training message is treated when it has no weights and is assigned to some categories.

As I mentioned at ETL 7.0, I didn't drop rows that were irrelevant because its possible that irrelevant rows have been added before selecting the data from database. 

So here at this step the data needs to be cleaned either way. My following steps:
- remove column 'original' as there are many missing values as explored during ETL and cannot be proven here as you can see above ^^
- remove rows when message starts with 'NOTES:'
- remove rows with related not in [0, 1]
- remove rows that have no category at all
- remove rows with messages that are much longer than the length of all messages (I intend to predict messages, no prosa texts)
- remove rows that would be empty after message has gone through Tokenize( ) so please allow me at this place to slightly change the order of to do's.

In [10]:
df = dsh.RemoveColumnsByWildcard(df, 'original')

-------------------- Start finding and removing columns matchting to wildcards: ['original'] --------------------
Columns found to remove:  ['original']
-------------------- Finished removing columns matchting to wildcards --------------------


In [11]:
# Just keep messages with related = 0 or 1 (drop 2 or potentially other values)

df = dsh.SelectRowsWithValueInColumn(df, 'related', [0, 1])

188 rows (ca. 0.72%) have been removed not having value/s "[0, 1]" in column "related"


In [12]:
# # Remove rows that have no category at all

# # Get all categories
# colsToSum = list(dsh.RemoveColumnsByWildcard(df, ['index', 'id', 'message', 'original', 'genre']))

# shapeBefore = df.shape
# dfRemovedNoCategory = df[df[colsToSum].sum(axis = 1) == 0]
# df = df[df[colsToSum].sum(axis = 1) > 0]

# print('Rows removed that had no category: ', shapeBefore[0] - df.shape[0])

In [13]:
# Remove messages that are much longer than the average

#df = dsh.RemoveRowsByValuesOverAverage(df, 'message', 4)

### 2. Write a tokenization function to process your text data

In [14]:
class TokenizerSettings:
    
    SupportedLanguages = [
    'english', 
    #'frensh',
    #'haitian'
    ]
    
    ConsiderOnlyLetters = 0
    ConsiderOnlyLettersNumbers = 0
    ConsiderMinimumLength = 0
    
    ReplaceUrlWithPlaceHolder = False
    CleanUrlReg = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    UrlPlaceHolder = ''    
    
    RemovePunctuation = True
    RemovePuncReg = r'[^a-zA-Z0-9]'
    
    UseLemmatizer = 0
    UseStemmer = 1
    UseStopWords = 1
    UseTokenizer = 1
    UseWordNet = 1
    
    Lemmatizer = WordNetLemmatizer()
    Stemmer = PorterStemmer()
    
    # If using the option UseWordNet:
    
    # By hazard I saw a message with that organization/institution,
    # filtered roughtly 40 rows and decided to consider/give this word a weight
    # and not just drop it.
    # So I intended to make it parameterizable.
    WhiteListWords = set(['UNHCR'])
    

    
def tokenize(text):
    
    text = text.lower() # I prefer calling this function once instead for each word
    
    if TokenizerSettings.ReplaceUrlWithPlaceHolder:
        if not dsh.IsNullOrEmpty(TokenizerSettings.CleanUrlReg):
            foundUrls = re.findall(TokenizerSettings.CleanUrlReg, text)

            for url in foundUrls:
                text = text.replace(url, TokenizerSettings.UrlPlaceHolder)
    
    if TokenizerSettings.RemovePunctuation:
        text = re.sub(TokenizerSettings.RemovePuncReg, ' ', text.lower())
    
    tokens = word_tokenize(text)

    # Call str() if values have been passed to whitelistwords that are no strings
    whiteList = [str(white).lower() for white in TokenizerSettings.WhiteListWords]
    
    cleanTokens = []
    for tok in tokens:

        if TokenizerSettings.ConsiderOnlyLetters:
            tok = dsh.KeepLetters(tok)
        elif TokenizerSettings.ConsiderOnlyLettersNumbers:
            tok = dsh.KeepLettersNumbers(tok)
            
        if TokenizerSettings.UseLemmatizer:
            tok = TokenizerSettings.Lemmatizer.lemmatize(tok)
        
        if TokenizerSettings.UseStemmer:
            stemmed = TokenizerSettings.Stemmer.stem(tok)
        
        if len(tok) < TokenizerSettings.ConsiderMinimumLength:
            #print('Too short')
            continue
        
        isStopWord = False
        if TokenizerSettings.UseStopWords:
            try:
                for lang in list([lng for lng in TokenizerSettings.SupportedLanguages if lng is not None and len(lng) > 0]):
                    if tok in stopwords.words(lang):
                        isStopWord = True
                        break
            except:
                print('Error during stop word handling with language: ', lang, \
                    '\nand token: ', tok)
        if isStopWord:
            #print('Stopword')
            continue
        
        isUnknownWord = False
        if TokenizerSettings.UseWordNet:
            if not wordnet.synsets(tok):
                if not tok in whiteList:
                    isUnknownWord = True
        if isUnknownWord:
            continue
        
        cleanTokens.append(tok)
    
    return cleanTokens

So these 37 "useless" messages can smoothly be dropped.

In [15]:
X = dsh.SelectColumnsByWildcard(df, ['message'])

Y = dsh.RemoveColumnsByWildcard(df, ['id', 'message', 'original', 'genre'])

-------------------- Start finding and keeping columns matchting to wildcards: ['message'] --------------------
-------------------- Finished keeping columns matchting to wildcards: 1 --------------------
-------------------- Start finding and removing columns matchting to wildcards: ['id', 'message', 'original', 'genre'] --------------------
Columns found to remove:  ['id', 'message', 'genre']
-------------------- Finished removing columns matchting to wildcards --------------------


In [16]:
Y.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [17]:
_cv = CountVectorizer(tokenizer = tokenize)
_tfidf = TfidfTransformer()

_classifierInner = RandomForestClassifier(
    #n_estimators = 10,
    #min_samples_split = 10,
#    random_state = 42
)
_classifierOuter = MultiOutputClassifier(_classifierInner)

pipeline = Pipeline([('vect', _cv),
                     ('tfidf', _tfidf),
                     ('clf', _classifierOuter,)
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [18]:
XTrain, XTest, yTrain, yTest = dsh.SplitDataTrainTest(X, Y, randomState = 21)

-------------------- Test/Train split (test size = 0.3, random state = 21: --------------------
Training: X = (18219, 1), y = (18219, 36)
Test    : X = (7809, 1), y = (7809, 36)
-------------------- - --------------------


In [19]:
np.random.seed(63)
model = dsh.TrainModel(pipeline, XTrain['message'].values, yTrain.values)

-------------------- Start fitting model to data --------------------
-------------------- Train time: 0:02:21.389836 --------------------


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [20]:
def TestModel(model, xTest, yTest, columns, printIncorrect = False):
    
    yPred = model.predict(xTest)
    
    dicClassifyReports = {}
    
    for ind in range(yPred.shape[1]):
        colName = columns[ind]
        dsh.PrintLine('Column: ' + str(colName))
        cr = classification_report(yTest[:, ind], yPred[:, ind])
        dicClassifyReports[colName] = cr
        print(cr)
        
#         cntErr = 0
#         dirError = {}
#         lstFalses = []
#         for cind in range(yTest.shape[1]):

#             cntErr = 0
#             colname = columns[cind]

#             for ind in range(yTest.shape[0]):

#                 if yTest[ind, cind] != yPred[ind, cind]:
#                     cntErr += 1
#                     lstFalses.append(str(colname) + ' - ' + str(xTest[ind]))

#             err = cntErr * 100 / yTest.shape[0]
#             dirError[str(colname)] = err

#         dfError = pd.DataFrame({'Errors' : list(dirError.values())}, index = columns)
#     if printIncorrect:
#         for val in lstFalses:
#             print(tokenize(val))
#         print('False predicted')

#     dfError.sort_values('Errors',ascending = False)
        
#     dsh.PrintLine()
    
#     return dfError

In [21]:
TestModel(model, XTest['message'].values, yTest.values, yTest.columns, False)

-------------------- Column: related --------------------
             precision    recall  f1-score   support

          0       0.62      0.43      0.51      1897
          1       0.83      0.92      0.87      5912

avg / total       0.78      0.80      0.78      7809

-------------------- Column: request --------------------
             precision    recall  f1-score   support

          0       0.90      0.97      0.93      6476
          1       0.76      0.48      0.59      1333

avg / total       0.88      0.89      0.87      7809

-------------------- Column: offer --------------------
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7779
          1       0.00      0.00      0.00        30

avg / total       0.99      1.00      0.99      7809

-------------------- Column: aid_related --------------------
             precision    recall  f1-score   support

          0       0.76      0.85      0.80      4582
          1   

C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [22]:
def performance_metric(y_true, y_pred):
    """Calculate median F1 score for all of the output classifiers

        Args:
        y_true: array. Array containing actual labels.
        y_pred: array. Array containing predicted labels.

        Returns:
        score: float. Median F1 score for all of the output classifiers
        """
    f1_list = []
    for i in range(np.shape(y_pred)[1]):
        f1 = f1_score(np.array(y_true)[:, i], y_pred[:, i])
        f1_list.append(f1)
        
    score = np.median(f1_list)
    return score

In [23]:
parameters = {
    
    'clf__estimator__min_samples_split' : [2, 5, 10],
    'clf__estimator__n_estimators' : [10, 15, 20],
    'tfidf__use_idf':[True], # definitely use it
    'vect__min_df': [1, 5],
    
#    'clf__estimator__kernel': ['poly'], 
#    'clf__estimator__degree': [1, 2, 3],
#    'clf__estimator__C':[1, 10, 100],
    
#    'clf__estimator__max_features' : [ 'auto', 'sqrt', 'log2'],
#    'clf__estimator__max_depth' : [2, 4, 8],
#    'clf__estimator__min_samples_split' : [10, 15, 20],
#    'clf__estimator__min_samples_leaf' : [2, 5, 10],
#    'clf__estimator__bootstrap' : [True, False],
#    'clf__n_jobs' : [4],
#    'clf__estimator__n_jobs' : [4],
#    'clf__estimator__random_state' : [42],
}

scorer = make_scorer(performance_metric)

cv = GridSearchCV(
    pipeline, 
    param_grid = parameters,
    
    # doc: Controls the verbosity: 
    #      the higher, the more messages.
    verbose = 10 ,
    
    scoring = scorer
)

In [ ]:
np.random.seed(105)
impModel = cv.fit(XTrain['message'].values, yTrain.values)

Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.12356042678623323, total= 2.1min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.2min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.13484848484848486, total= 2.0min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  6.4min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.1702638818925098, total= 2.0min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 


...

[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=15, tfidf__use_idf=True, vect__min_df=1, score=0.134006734006734, total= 2.2min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=15, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed: 25.6min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=15, tfidf__use_idf=True, vect__min_df=1, score=0.16641367452501024, total= 2.2min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=15, tfidf__use_idf=True, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 29.0min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=15, tfidf__use_idf=True, vect__min_df=5, score=0.2079231692677071, total= 2.0min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=15, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=15, tfidf__use_idf=True, vect__min_df=5, score=0.19900803410395146, total= 2.0min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=15, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=15, tfidf__use_idf=True, vect__min_df=5, score=0.2188501095936604, total= 2.0min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20, tfidf__use_idf=True, vect__min_df=1, score=0.09290398288120384, total= 2.4min
[CV] clf__estimator__min_samples_split=2, clf__est

In [ ]:
impModel.cv_results_

In [ ]:
impModel.best_params_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
TestModel(impModel, XTest['message'].values, yTest.values, yPred, yTest.columns)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

When going through algorithms http://scikit-learn.org/stable/modules/multiclass.html I saw SVC and remembered that supported vector machines project data in higher spaces to separate data with an hyperplane. 
As bag of words is a large vector, I intend to find out if following algorithm works better:

In [ ]:
_cv = CountVectorizer(tokenizer = tokenize)
_tfidf = TfidfTransformer()
_classifierInner = SVC()
_classifierOuter = MultiOutputClassifier(_classifierInner)


pipeline2 = Pipeline([('vect', _cv),
                     ('tfidf', _tfidf),
                     ('clf', _classifierOuter,)
                    ])

parameters2 = {
    
#    'clf__estimator__min_samples_split' : [2, 5, 10],
#    'clf__estimator__n_estimators' : [10, 15, 20],
    'tfidf__use_idf':[True], # definitely use it
    'vect__min_df': [1, 5],
    
    'clf__estimator__kernel': ['poly'], 
    'clf__estimator__degree': [1, 2, 3],
    'clf__estimator__C':[1, 10, 50],
    
#    'clf__estimator__max_features' : [ 'auto', 'sqrt', 'log2'],
#    'clf__estimator__max_depth' : [2, 4, 8],
#    'clf__estimator__min_samples_split' : [10, 15, 20],
#    'clf__estimator__min_samples_leaf' : [2, 5, 10],
#    'clf__estimator__bootstrap' : [True, False],
#    'clf__n_jobs' : [4],
#    'clf__estimator__n_jobs' : [4],
#    'clf__estimator__random_state' : [42],
}

In [ ]:
cv2 = GridSearchCV(
    pipeline2, 
    param_grid = parameters2,
    
    # doc: Controls the verbosity: 
    #      the higher, the more messages.
    verbose = 10 ,
    
    scoring = scorer
)

np.random.seed(84)

impModel2 = cv.fit(XTrain['message'].values, yTrain.values)

In [ ]:
impModel2.cv_results_

### 9. Export your model as a pickle file

In [ ]:
#https://stackoverflow.com/questions/34143829/sklearn-how-to-save-a-model-created-from-a-pipeline-and-gridsearchcv-using-jobli
#If you want to dump your object into one file - use:  

joblib.dump(bestEstimator, DisasterConfig.SaveFileJoblib, compress = 1)
#pickle.dump(tuned_model, open(DisasterConfig.SaveFilePickle, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.